## Gathering Docs

### Run the following command in the jupyter terminal to clone MDN glossary repo: git clone https://github.com/mdn/content.git

In [1]:
import os 

print(os.getcwd())

/home/jovyan/TechGlossary/TechGlossary


In [ ]:
from pathlib import Path
mdn_glossary_path = Path(f"{os.getcwd()}/content/files/en-us/glossary")

# veriyfing that all the files are there
for item in mdn_glossary_path.iterdir():
    print(item)

## Step 1. Parse MDN glossary repo

In [3]:
%pip install markdown

  Using cached markdown-3.10-py3-none-any.whl.metadata (5.1 kB)
Using cached markdown-3.10-py3-none-any.whl (107 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


%pip install python-frontmatter

In [12]:
#To investigate content of a single document
import frontmatter
import pandas as pd
import markdown
from bs4 import BeautifulSoup

term = "png"
with open(f"{os.getcwd()}/content/files/en-us/glossary/{term}/index.md", "r") as f:
    content = f.read()
    post = frontmatter.loads(content)
    
    # Convert Markdown to HTML
    html = markdown.markdown(post.content)
    
    # Extract plain text from HTML
    plain_text = BeautifulSoup(html, "html.parser").get_text()
    
    # Create a single-row DataFrame with only the content
    df = pd.DataFrame([plain_text], columns=["content"])
    
    print(df)

                                                                                                                                                                                                                                                                                                                                                                                      content
0  PNG (Portable Network Graphics) is a graphics file format that supports lossless data compression.\nSee also\n\nPNG section in the Image file type and format guide\nOther image formats (glossary terms): {{Glossary("GIF")}}, {{Glossary("JPEG")}}, {{Glossary("SVG")}}, {{Glossary("WebP")}}\nHTML images (Learn web development)\nAdding custom icons to your site\nPNG on Wikipedia\n


## Parsing the MDN Repo

In [18]:
# Creating df to build pyterrier index with html for later display 
import os
import pandas as pd
import frontmatter
import re
import markdown
from bs4 import BeautifulSoup

def load_glossary_for_pyterrier(glossary_root: str) -> pd.DataFrame:
    """
    Iterate through all subfolders in the glossary folder,
    read each Markdown file, clean text for indexing, replace
    all double-bracketed shortcodes with just the first term,
    remove Markdown formatting, strip See also sections,
    and return a DataFrame with columns:
    - docno: unique document identifier
    - text: cleaned plain text for search/indexing
    """
    rows = []

    # Regex to match any {{Shortcode("term", ...)}}
    double_bracket_regex = re.compile(
        r'\{\{\s*[\w]+\(\s*["\'](.*?)["\'].*?\)\s*\}\}', re.IGNORECASE
    )

    # Regex to remove See also section (from 'See also' to end of text)
    see_also_regex = re.compile(r'See also.*', re.IGNORECASE | re.DOTALL)

    for term_folder in os.listdir(glossary_root):
        folder_path = os.path.join(glossary_root, term_folder)
        if os.path.isdir(folder_path):
            md_file = os.path.join(folder_path, "index.md")
            if os.path.exists(md_file):
                try:
                    # Load Markdown with frontmatter
                    post = frontmatter.load(md_file)

                    # Replace any {{Shortcode(...)}} with just the first argument
                    content_no_placeholders = double_bracket_regex.sub(r'\1', post.content)

                    # Remove "See also" section
                    content_no_see_also = see_also_regex.sub("", content_no_placeholders)

                    # Convert Markdown → HTML
                    html_content = markdown.markdown(content_no_see_also)

                    # Convert HTML → plain text (removes **bold**, links, code formatting, etc.)
                    plain_text = BeautifulSoup(html_content, "html.parser").get_text()

                    # Clean up extra whitespace
                    clean_text = re.sub(r"\s+", " ", plain_text).strip()

                    # Assign docno (using folder name)
                    docno = term_folder

                    # Add row to DataFrame
                    rows.append({
                        "docno": docno,
                        "text": clean_text
                    })

                except Exception as e:
                    print(f"Error processing {md_file}: {e}")

    df = pd.DataFrame(rows, columns=["docno", "text"])
    return df

# Usage
glossary_root = f"{os.getcwd()}/content/files/en-us/glossary"
df = load_glossary_for_pyterrier(glossary_root)

pd.set_option("display.max_colwidth", None)
print(df)

                    docno  \
0                 closure   
1                   cache   
2              decryption   
3           type_coercion   
4                     ril   
..                    ...   
597              copyleft   
598          cryptography   
599                xforms   
600  flow_relative_values   
601                  host   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [66]:
%pip install -U python-terrier jpype1

  Using cached jpype1-1.6.0-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.0 kB)
  Using cached pyjnius-1.7.0-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (13 kB)
Using cached jpype1-1.6.0-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (496 kB)
Using cached pyjnius-1.7.0-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (1.7 MB)

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [68]:
# imports
import os, pyterrier as pt

# Point PyTerrier to your freshly installed JDK
os.environ["JAVA_HOME"] = os.path.expanduser("~/jdk/jdk-17.0.8+7")
os.environ["JVM_PATH"]  = os.path.join(os.environ["JAVA_HOME"], "lib/server/libjvm.so")
os.environ["PATH"]      = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

# Confirm libjvm.so exists
!ls $JVM_PATH

# Initialize PyTerrier
if not pt.java.started():
     pt.java.init()

print(" PyTerrier initialized successfully with local JDK.")

/home/jovyan/jdk/jdk-17.0.8+7/lib/server/libjvm.so
 PyTerrier initialized successfully with local JDK.


Java started and loaded: pyterrier.java.colab, pyterrier.java, pyterrier.java.24, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


## Step 2. Pre-process 
Preprocessing choices determine what the model can “see.” We will adopt a minimal but effective pipeline:

- Lowercasing to normalize case.
- Tokenization on alphabetic words
- Stopword removal to reduce high-frequency function words.
- Stemming to conflate morphological variants (e.g., “running” → “run”).

In [19]:
%pip install nltk

  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
Using cached nltk-3.9.2-py3-none-any.whl (1.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.4/800.4 kB 22.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    '''
     Step 1: Convert the input to lowercase and extract valid words
             (e.g., alphabetic tokens of at least 3 letters).
     Hint: You can use regex (e.g., re.findall) for token extraction.

     Step 2: Remove common stop words
             (you can use a predefined list like NLTK's stopwords or your own).

     Step 3: Apply stemming or
             (use something like PorterStemmer or WordNetLemmatizer).

    Step 4: Join the cleaned tokens back into a single string.

     Return the cleaned text
     '''
    tokens = nltk.word_tokenize(text)
    tokens = [w.lower() for w in tokens if w.isalpha() and w not in stop_words]
    tokens = [stemmer.stem(w) for w in tokens]
    return " ".join(tokens)

NameError: name 'PorterStemmer' is not defined

## Clean text from parsed MDN repo and index

In [ ]:
df['clean_text'] = df['text'].apply(clean_text)

# Prepare documents for PyTerrier
docs_for_index = [{"docno": row["docno"], "text": row["clean_text"]} for _, row in df.iterrows()]

# Build index
indexer = pt.IterDictIndexer("./glossary_index")  # folder to store index
index_ref = indexer.index(docs_for_index)

print("Indexing complete! Your glossary is ready for retrieval.")